# Things to install

## Colab

In [ ]:
!apt update && apt install -y openslide-tools
!pip install openslide-python

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Locale

In [ ]:
!pip install openslide-python
!pip install opencv-python
!pip install imgaug
!pip install scikit-learn

In [ ]:
!pip install scikit-learn

In [1]:
# The path can also be read from a config file, etc.
OPENSLIDE_PATH = r'C:\Users\sofia\openslide-win64-20230414\openslide-win64-20230414\bin'

import os
if hasattr(os, 'add_dll_directory'):
    # Python >= 3.8 on Windows
    with os.add_dll_directory(OPENSLIDE_PATH):
        import openslide
else:
    import openslide

# Code

# Patches extrapolation from wsi

## - Import & constants

In [ ]:
import concurrent.futures
import glob
import numpy as np
from thread import process_svs_file
import threading
import openslide
import cv2
from matplotlib import pyplot as plt
import xml.etree.ElementTree as ET

path_to_images = "../slides/"
path_to_annotations = "../annotations/"
el_width = 2000
el_height = 2000
output_width = 1000
output_height = 1000

## - Function

In [ ]:
def get_annotatios(file_path):
    # Parsa il file XML delle annotazioni
    tree = ET.parse(file_path)
    root = tree.getroot()
    # Ottieni tutte le annotazioni dal file XML
    annotations = []
    for annotation in root.iter('Annotation'):
        name = annotation.get('Name')
        coordinates = []
        for coordinate in annotation.iter('Coordinate'):
            x = float(coordinate.get('X'))
            y = float(coordinate.get('Y'))
            coordinates.append((x, y))
        annotations.append({'name': name, 'coordinates': coordinates})
    return annotations

In [ ]:
def is_mostly_white(image, threshold_w=0.85, threshold_p=0.98):
    # Converti l'immagine in scala di grigi
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Calcola la soglia per considerare i pixel bianchi
    pixel_threshold = int(threshold_w * 255)

    # Conta i pixel bianchi nell'immagine
    white_pixels = np.sum(gray_image >= pixel_threshold)

    # Calcola la percentuale di pixel bianchi rispetto alla dimensione totale dell'immagine
    white_percentage = white_pixels / \
        (gray_image.shape[0] * gray_image.shape[1])

    # Verifica se la percentuale di pixel bianchi supera la soglia
    if white_percentage >= threshold_p:
        return True, white_percentage
    else:
        return False, white_percentage

In [ ]:
def get_labels(labels, annotations):
    for annotation in annotations:
        polygon = np.array([annotation['coordinates']], dtype=np.int32)
        cv2.fillPoly(labels, polygon, 1)

In [ ]:
def plt_image(image, labes):
    fig, axs = plt.subplots(1, 2)
    # Primo subplot: labels
    axs[0].imshow(labes)
    axs[0].axis('off')
    # Secondo subplot: immagine
    axs[1].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    axs[1].axis('off')
    # Mostra i subplot affiancati
    plt.show()

In [ ]:
def extrapolate_patches(wsi, annotation, el_width, el_height, output_width, output_height):
    # Ottieni le dimensioni dell'immagine
    w, h = wsi.dimensions
    label_image = np.zeros((h, w), dtype=np.uint8)
    annotations = get_annotatios(annotation)
    get_labels(label_image, annotations)

    # Calcola il numero di righe e colonne necessarie per suddividere l'immagine
    num_rows = h // el_height
    num_cols = w // el_width

    # Crea un'immagine di output con le stesse dimensioni dell'immagine svs

    dataset = []
    labels = []

    thread_name = threading.current_thread().name
    file = open("../log/thread_" + thread_name + ".txt", "a")
    file.write("Sto per leggere il file wsi\n")

    wsi = np.array(wsi.read_region((0, 0), 0, (w, h)))

    file.write("File letto\n")
    for row in range(num_rows):
        for col in range(num_cols):
            # for row in range(3, 5):
            #    for col in range(58, 60):
            # Calcola le coordinate di inizio e fine per l'immagine corrente
            x = col * el_width
            y = row * el_height
            x_end = x + el_width
            y_end = y + el_height

            # Estrai l'immagine corrente
            region = wsi[y: y_end, x: x_end]
            image = cv2.cvtColor(region, cv2.COLOR_RGBA2BGR)

            is_white, p = is_mostly_white(image)
            if not is_white:
                r_image = cv2.resize(
                    image, (output_width, output_height), interpolation=cv2.INTER_CUBIC)
                r_label_image = cv2.resize(
                    label_image[y:y_end, x: x_end], (output_width, output_height), interpolation=cv2.INTER_CUBIC)

                dataset.append(r_image)
                labels.append(r_label_image)
                if not ((col == num_cols-1) or (row == num_rows-1)):
                    x_h = x + el_width // 2
                    x_v = x
                    x_d = x + el_width // 2
                    y_h = y
                    y_v = y + el_height // 2
                    y_d = y + el_height // 2
                    region_h = wsi[y_h: y_h + el_height,
                                   x_h: x_h + el_width]
                    region_v = wsi[y_v: y_v + el_height,
                                   x_v: x_v + el_width]
                    region_d = wsi[y_d: y_d + el_height,
                                   x_d: x_d + el_width]
                    image_h = cv2.cvtColor(
                        np.array(region_h), cv2.COLOR_RGBA2BGR)
                    image_v = cv2.cvtColor(
                        np.array(region_v), cv2.COLOR_RGBA2BGR)
                    image_d = cv2.cvtColor(
                        np.array(region_d), cv2.COLOR_RGBA2BGR)
                    is_white_h, _ = is_mostly_white(image_h)
                    is_white_v, _ = is_mostly_white(image_v)
                    is_white_d, _ = is_mostly_white(image_d)
                    if not is_white_h:
                        r_image = cv2.resize(
                            image_h, (output_width, output_height), interpolation=cv2.INTER_CUBIC)
                        r_label_image = cv2.resize(
                            label_image[y_h: y_h+el_height, x_h: x_h+el_width], (output_width, output_height), interpolation=cv2.INTER_CUBIC)

                        dataset.append(r_image)
                        labels.append(r_label_image)

                    if not is_white_v:
                        r_image = cv2.resize(
                            image_v, (output_width, output_height), interpolation=cv2.INTER_CUBIC)
                        r_label_image = cv2.resize(
                            label_image[y_v: y_v+el_height, x_v: x_v+el_width], (output_width, output_height), interpolation=cv2.INTER_CUBIC)

                        dataset.append(r_image)
                        labels.append(r_label_image)

                    if not is_white_d:
                        r_image = cv2.resize(
                            image_d, (output_width, output_height), interpolation=cv2.INTER_CUBIC)
                        r_label_image = cv2.resize(
                            label_image[y_d: y_d+el_height, x_d: x_d+el_width], (output_width, output_height), interpolation=cv2.INTER_CUBIC)

                        dataset.append(r_image)
                        labels.append(r_label_image)

    file.write("Wsi elaborato\nDataset di dimensione:" +
               str(np.array(dataset).shape) + "\nLabels di dimensione:" + str(np.array(labels).shape))
    file.close()
    return dataset, labels


In [ ]:
def process_svs_file(svs_file, path_to_annotations, path_to_images, el_width, el_height, output_width, output_height):
    thread_name = threading.current_thread().name
    file = open("../log/thread_" + thread_name + ".txt", "x")
    file.write("Sono il thread" + thread_name + "\n")
    file.write("Sto elaborando il file " +
               svs_file[len(path_to_images):-4] + "\n")
    file.close()
    # Ottieni il percorso del file .xml corrispondente
    annotation = path_to_annotations + \
        svs_file[len(path_to_images):-4] + ".xml"
    # Carica l'immagine svs
    wsi = openslide.OpenSlide(svs_file)
    d, l = extrapolate_patches(
        wsi, annotation, el_width, el_height, output_width, output_height)
    np.save('../slides/' +
            svs_file[len(path_to_images):-4] + '.npy', np.array(d))
    np.save('../annotations/' +
            svs_file[len(path_to_images):-4] + '_label.npy', np.array(l))
    return d, l

## - Code


In [ ]:
dataset = []
labels = []

file = open("../log/job.txt", "x")
file.write("Il task è iniziato\n")
file.close()

# Ottieni la lista dei file .svs nella cartella slides
svs_files = glob.glob(path_to_images + "*.svs")

# Creazione di un ThreadPoolExecutor con un numero di thread desiderato
num_threads = 9  # Numero di thread da utilizzare
executor = concurrent.futures.ThreadPoolExecutor(max_workers=num_threads)


In [ ]:
# Lista per salvare i future restituiti dalle chiamate asincrone
futures = []
file = open("../log/job.txt", "a")
file.write("Lancio i thread\n")

# Esecuzione della funzione extrapolate_patches in parallelo per ogni svs_file
for svs_file in svs_files:
    future = executor.submit(process_svs_file, svs_file, svs_file, path_to_annotations,
                             path_to_images, el_width, el_height, output_width, output_height)
    futures.append(future)

file.write("Aspetto la fine dei thread\n")
# Attendere il completamento di tutte le chiamate asincrone
concurrent.futures.wait(futures)

file.write("I thread hanno finito, concateno i risultati\n")
# Ottenere i risultati dai future
dataset = []
labels = []
for future in futures:
    d, l = future.result()
    dataset.extend(d)
    labels.extend(l)

In [ ]:
dataset = np.array(dataset)
labels = np.array(labels)

np.save('../slides/dataset.npy', dataset)
np.save('../annotations/labels.npy', labels)

file.write("Risultati salvati\n")

file.close()

# Preprocessing data and data augmentation

## - Import and constants

In [1]:
import imgaug.augmenters as iaa
import tensorflow as tf
from imgaug.augmentables.segmaps import SegmentationMapsOnImage
from sklearn.model_selection import train_test_split
import numpy as np

path_to_dataset = "../dataset/slides/dataset.npy"
path_to_labels = "../dataset/annotations/labels.npy"

## - Function

In [ ]:
def data_augment():
    return iaa.Sequential([
        iaa.Dropout((0, 0.05)),  # Remove random pixel
        iaa.Affine(rotate=(-30, 30)),  # Rotate between -30 and 30 degreed
        iaa.Fliplr(0.5),  # Flip with 0.5 probability
        iaa.Crop(percent=(0, 0.2), keep_size=True),  # Random crop
        # Add -50 to 50 to the brightness-related channels of each image
        iaa.WithBrightnessChannels(iaa.Add((-50, 50))),
        # Change images to grayscale and overlay them with the original image by varying strengths, effectively removing 0 to 50% of the color
        iaa.Grayscale(alpha=(0.0, 0.5)),
        # Add random value to each pixel
        iaa.GammaContrast((0.5, 2.0), per_channel=True),
        # Local distortions of images by moving points around
        iaa.PiecewiseAffine(scale=(0.01, 0.1)),
    ], random_order=True)

In [ ]:
def process_data(image, label):
    return tf.cast(image, tf.float32)/255, tf.one_hot(label, 2, name="label", axis=-1)

In [ ]:
def data_aug_impl(dataset, image_train, label_train):
    da = data_augment()
    segmented_label_train = [SegmentationMapsOnImage(
        label, shape=dataset[1].shape) for label in label_train]
    image_train_copy = image_train.copy()
    for _ in range(1):
        augmented_images, augmented_labels = da(
            images=image_train_copy, segmentation_maps=segmented_label_train)
        image_train = np.append(image_train, augmented_images, axis=0)
        label_train = np.append(label_train, np.array(
            [label.get_arr() for label in augmented_labels]), axis=0)

    return image_train, label_train

In [ ]:
def generate_train_data_tensor(image_train, label_train):
    train_data = tf.data.Dataset.from_tensor_slices((image_train, label_train))
    train_data = train_data.map(
        process_data, num_parallel_calls=tf.data.AUTOTUNE)
    train_data = train_data.cache()
    train_data = train_data.shuffle(100)
    train_data = train_data.batch(128)
    train_data = train_data.prefetch(tf.data.AUTOTUNE)
    return train_data

## - Code

In [ ]:
print("Loading dataset and labels")

dataset = np.load(path_to_dataset)[0:10]
labels = np.load(path_to_labels)[0:10]

print(
    f"Dataset and labels loaded\nDataset shape {dataset.shape} \nLabels shape {labels.shape}")

In [ ]:
image_train, image_test, label_train, label_test = train_test_split(
    dataset, labels, test_size=0.25, random_state=42)

print("Dataset and labels splitted in train and test set\n" +
      f"image_train shape {image_train.shape} - label_train shape {label_train.shape}" +
      f"image_test shape {image_test.shape} - image_test shape {label_test.shape}")

In [ ]:
image_train, label_train = data_aug_impl(dataset, image_train, label_train)

print("Applied data agumentation to train set\n" +
      f"image_train augmented shape {image_train.shape} - label_train augmented shape {label_train.shape}")

In [ ]:
train_data = generate_train_data_tensor(image_train, label_train)

print("Preprocessed and created tensor dataset")

# SegNet

## - Import and constants

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization, Conv2D, UpSampling2D

NUM_CLASSES = 2
INPUT_SHAPE = (400, 400, 3)
IMAGE_SIZE = (400, 400)

## - Class

In [ ]:
class SegNet(tf.keras.models.Model):
    def __init__(self):
        super().__init__()
        vgg19 = tf.keras.applications.vgg19.VGG19(
            include_top=False,   # Exclusion of the last 3 layers
            weights='imagenet',
            # input_tensor=None,
            input_shape=INPUT_SHAPE,
            pooling='max',
            classes=NUM_CLASSES,
            classifier_activation='relu'
        )
        self.encoder = tf.keras.Sequential([
            # vgg19.get_layer('input_2'),
            # vgg19.get_layer('input_1'),
            BatchNormalization(),
            vgg19.get_layer('block1_conv1'),
            BatchNormalization(),
            vgg19.get_layer('block1_conv2'),
            BatchNormalization(),
            vgg19.get_layer('block1_pool'),
            vgg19.get_layer('block2_conv1'),
            BatchNormalization(),
            vgg19.get_layer('block2_conv2'),
            BatchNormalization(),
            vgg19.get_layer('block2_pool'),
            vgg19.get_layer('block3_conv1'),
            BatchNormalization(),
            vgg19.get_layer('block3_conv2'),
            BatchNormalization(),
            vgg19.get_layer('block3_conv3'),
            BatchNormalization(),
            vgg19.get_layer('block3_conv4'),
            BatchNormalization(),
            vgg19.get_layer('block3_pool'),
            vgg19.get_layer('block4_conv1'),
            BatchNormalization(),
            vgg19.get_layer('block4_conv2'),
            BatchNormalization(),
            vgg19.get_layer('block4_conv3'),
            BatchNormalization(),
            vgg19.get_layer('block4_conv4'),
            BatchNormalization(),
            vgg19.get_layer('block4_pool')
        ])

        self.decoder = tf.keras.Sequential([
            # Block 5
            UpSampling2D(size=(2, 2)),
            Conv2D(512, (3, 3), activation='relu',
                   padding='same'),  # TODO check padding
            BatchNormalization(),
            Conv2D(512, (3, 3), activation='relu', padding='same'),
            BatchNormalization(),
            Conv2D(512, (3, 3), activation='relu', padding='same'),
            BatchNormalization(),
            Conv2D(512, (3, 3), activation='relu', padding='same'),
            BatchNormalization(),
            # Block 4
            UpSampling2D(size=(2, 2)),
            Conv2D(512, (3, 3), activation='relu', padding='same'),
            BatchNormalization(),
            Conv2D(512, (3, 3), activation='relu', padding='same'),
            BatchNormalization(),
            Conv2D(512, (3, 3), activation='relu', padding='same'),
            BatchNormalization(),
            Conv2D(512, (3, 3), activation='relu', padding='same'),
            BatchNormalization(),
            # Block 3
            UpSampling2D(size=(2, 2)),
            Conv2D(256, (3, 3), activation='relu', padding='same'),
            BatchNormalization(),
            Conv2D(256, (3, 3), activation='relu', padding='same'),
            BatchNormalization(),
            Conv2D(256, (3, 3), activation='relu', padding='same'),
            BatchNormalization(),
            Conv2D(256, (3, 3), activation='relu', padding='same'),
            BatchNormalization(),
            # Block 2
            UpSampling2D(size=(2, 2)),
            Conv2D(128, (3, 3), activation='relu', padding='same'),
            BatchNormalization(),
            Conv2D(128, (3, 3), activation='relu', padding='same'),
            BatchNormalization(),
            # Block 1
            UpSampling2D(size=(2, 2)),
            Conv2D(64, (3, 3), activation='relu', padding='same'),
            BatchNormalization(),
            Conv2D(64, (3, 3), activation='relu', padding='same'),
            BatchNormalization(),
            # Softmax
            Conv2D(NUM_CLASSES, (1, 1), activation='softmax', padding='valid'),
        ])

    def call(self, input):
        output = self.decoder(self.encoder(input))
        resized_output = tf.image.resize(output, IMAGE_SIZE)
        return resized_output

## - Code 

In [ ]:
segnet = SegNet()
print("Created segnet model")

optimizer = tf.keras.optimizers.legacy.SGD(learning_rate=0.01)
segnet.compile(optimizer=optimizer,
               loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:

print("Starting training")
history = segnet.fit(train_data, epochs=3)
print("Training completed")